# Scarico i dati

In [1]:
%%bash
# Prima scarico i dati
# le segnalazioni aperte
curl -sL 'http://www.rapspaservizi.it/develop1/bastaundito/bastaundito_dev/mappa' -H 'Connection: keep-alive' \
-H 'Cache-Control: max-age=0' -H 'Origin: http://www.rapspaservizi.it' -H 'Upgrade-Insecure-Requests: 1' \
-H 'Content-Type: application/x-www-form-urlencoded' \
-H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.40 Safari/537.36' \
-H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' \
-H 'Referer: http://www.rapspaservizi.it/develop1/bastaundito/bastaundito_dev/mappa' -H 'Accept-Encoding: gzip, deflate' \
-H 'Accept-Language: it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7' \
--data 'giorno_dal=01%2F01%2F2018&giorno_al=14%2F05%2F2018&attivita_id=31&circoscrizione_id=T&stato=A' >./aperte
# le segnalazioni aperte
curl -sL 'http://www.rapspaservizi.it/develop1/bastaundito/bastaundito_dev/mappa' -H 'Connection: keep-alive' \
-H 'Cache-Control: max-age=0' -H 'Origin: http://www.rapspaservizi.it' -H 'Upgrade-Insecure-Requests: 1' \
-H 'Content-Type: application/x-www-form-urlencoded' \
-H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.40 Safari/537.36' \
-H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' \
-H 'Referer: http://www.rapspaservizi.it/develop1/bastaundito/bastaundito_dev/mappa' -H 'Accept-Encoding: gzip, deflate' \
-H 'Accept-Language: it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7' \
--data 'giorno_dal=01%2F01%2F2018&giorno_al=14%2F05%2F2018&attivita_id=31&circoscrizione_id=T&stato=C' >./chiuse

## Estraggo le informazioni su ogni segnalazione

In [2]:
%%bash
# Poi le info su ogni pallino (le coordinate sono in un blocco diverso da quello anagrafico)
echo "data,indirizzo,foto,stato" >./dati.csv
## info su aperte
<aperte grep -P 'marker_[0-9]+.set.*' | \
perl -pe  's|(marker_)([0-9]+)(.*?)(<h[0-9]>)(.*?)(<.h[0-9]>)(<p>)(.*?)(<.p>)(.*?)(src=.")(.*?)(.")(.*)$|$5#$8#$12|g' | \
grep -v 'marker' | csvformat -d "#" | sed -e 's/$/,aperta/g' | sed -r 's/^\s+//' >>./dati.csv
## info su chiuse
<chiuse grep -P 'marker_[0-9]+.set.*' | \
perl -pe  's|(marker_)([0-9]+)(.*?)(<h[0-9]>)(.*?)(<.h[0-9]>)(<p>)(.*?)(<.p>)(.*?)(src=.")(.*?)(.")(.*)$|$5#$8#$12|g' | \
grep -v 'marker' | csvformat -d "#" | sed -e 's/$/,chiusa/g' | sed -r 's/^\s+//' >>./dati.csv

## Estraggo le coordinate di ogni segnalazione

In [3]:
%%bash
## info sulle coordinate
echo "latitude,longitude" >./coords.txt
<aperte grep -P 'google.maps.LatLng' | tail -n +3 | sed -r 's/^\s+//g' | perl -pe  's/^(.+?\()(.+?)(.;)/$2/g' >>coords.txt
<chiuse grep -P 'google.maps.LatLng' | tail -n +3 | sed -r 's/^\s+//g' | perl -pe  's/^(.+?\()(.+?)(.;)/$2/g' >>coords.txt
# metto insieme le info anagrafiche con quelle spaziali
paste -d , dati.csv coords.txt >./output.csv

## Metto insieme i dati estratti e creo output in CSV e GeoJSON

In [4]:
%%bash
# rimuovo i record che hanno 0 come coordinate e creo geojson
<output.csv csvgrep -c "latitude,longitude" -r '^.{0,1}0.{0,1}$' -i | csvjson --lat latitude --lon longitude >output.geojson

In [5]:
import pandas as pd
import geopandas as gpd

In [6]:
dati=pd.read_csv('output.csv')
dati.head(5)

,data,indirizzo,foto,stato,latitude,longitude
0,14/05/2018 17:20:36,"VIA SERRADIFALCO, 186",http://www.rapspaservizi.it/develop1/images/se...,aperta,38.126431,13.338010
1,14/05/2018 12:01:10,"PIAZZA BARTOLOMEO DA MESSINA, 2",http://www.rapspaservizi.it/develop1/images/se...,aperta,38.137805,13.363939
2,14/05/2018 09:42:58,"VIA SS.XL MARTIRI AL CASALOTTO, 6",http://www.rapspaservizi.it/develop1/images/se...,aperta,38.113358,13.362425
3,14/05/2018 08:57:02,"VIA ORSO MARIO CORBINO, 52",http://www.rapspaservizi.it/develop1/images/se...,aperta,38.105295,13.369227
4,14/05/2018 08:33:52,"VIA GIOTTO,",http://www.rapspaservizi.it/develop1/images/se...,aperta,38.131067,13.335596
